##Load bộ dữ liệu

In [ ]:
!git clone https://github.com/zliucr/CrossNER.git
%cd /content/CrossNER

Cloning into 'CrossNER'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 172 (delta 2), reused 2 (delta 2), pack-reused 164
Receiving objects: 100% (172/172), 2.30 MiB | 16.33 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/content/CrossNER


##Import modules, thư viện

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import numpy as np
import requests
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf

##Lấy các thẻ, ánh xạ dữ liệu,...

In [ ]:
def read_file(in_file_list):
  sentence,wt,word,tag = [],[],[],[]
  for in_file in in_file_list:
    with open(in_file) as f:
        for line in f.readlines():
            sp = line.strip().split('\n\n')
            word_tag = tuple(sp[0].split("\t"))
            if len(word_tag) > 1:
              word.append(word_tag[0])
              tag.append(word_tag[1])
              wt.append(tuple(word_tag))
            else:
              sentence.append(wt)
              wt = []
  word = list(set(word))
  word.append("ENDPAD")
  tag = list(set(tag))
  return sentence,word,tag
in_file_list_m = ["./ner_data/music/train.txt","./ner_data/music/test.txt","./ner_data/music/dev.txt"]
in_file_list_a = ["./ner_data/ai/train.txt","./ner_data/ai/test.txt","./ner_data/ai/dev.txt"]
in_file_list_p = ["./ner_data/politics/train.txt","./ner_data/politics/test.txt","./ner_data/politics/dev.txt"]
in_file_list_l = ["./ner_data/literature/train.txt","./ner_data/literature/test.txt","./ner_data/literature/dev.txt"]
in_file_list_s = ["./ner_data/science/train.txt","./ner_data/science/test.txt","./ner_data/science/dev.txt"]
sentence_m,words_m,tags_m = read_file(in_file_list_m)
sentence_a,words_a,tags_a = read_file(in_file_list_a)
sentence_p,words_p,tags_p = read_file(in_file_list_p)
sentence_l,words_l,tags_l = read_file(in_file_list_l)
sentence_s,words_s,tags_s = read_file(in_file_list_s)

In [ ]:
num_words_m = len(words_m)
num_words_a = len(words_a)
num_words_p = len(words_p)
num_words_l = len(words_l)
num_words_s = len(words_s)
num_tags_m = len(tags_m)
num_tags_a = len(tags_a)
num_tags_p = len(tags_p)
num_tags_l = len(tags_l)
num_tags_s = len(tags_s)

In [ ]:
word2idx_m = {w: i + 1 for i, w in enumerate(words_m)}
word2idx_a = {w: i + 1 for i, w in enumerate(words_a)}
word2idx_p = {w: i + 1 for i, w in enumerate(words_p)}
word2idx_l = {w: i + 1 for i, w in enumerate(words_l)}
word2idx_s = {w: i + 1 for i, w in enumerate(words_s)}
tag2idx_m = {t: i for i, t in enumerate(tags_m)}
tag2idx_a = {t: i for i, t in enumerate(tags_a)}
tag2idx_p = {t: i for i, t in enumerate(tags_p)}
tag2idx_l = {t: i for i, t in enumerate(tags_l)}
tag2idx_s = {t: i for i, t in enumerate(tags_s)}

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 100
def ToSequence(sentences,max_len,num_words,num_tags,word2idx,tag2idx):
  X = [[word2idx[w[0]] for w in s] for s in sentences]
  X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=num_words-1)
  y = [[tag2idx[w[1]] for w in s] for s in sentences]
  y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
  return X,y
X_m,y_m = ToSequence(sentence_m,max_len,num_words_m,num_tags_m,word2idx_m,tag2idx_m)
X_a,y_a = ToSequence(sentence_a,max_len,num_words_a,num_tags_a,word2idx_a,tag2idx_a)
X_p,y_p = ToSequence(sentence_p,max_len,num_words_p,num_tags_p,word2idx_p,tag2idx_p)
X_l,y_l = ToSequence(sentence_l,max_len,num_words_l,num_tags_l,word2idx_l,tag2idx_l)
X_s,y_s = ToSequence(sentence_s,max_len,num_words_s,num_tags_s,word2idx_s,tag2idx_s)

##Chia lại tập train, test

In [ ]:
from sklearn.model_selection import train_test_split
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split(X_m, y_m, test_size=0.05, random_state=1)
x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size=0.05, random_state=1)
x_train_p, x_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size=0.05, random_state=1)
x_train_l, x_test_l, y_train_l, y_test_l = train_test_split(X_l, y_l, test_size=0.05, random_state=1)
x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(X_s, y_s, test_size=0.05, random_state=1)

##Xây dựng mô hình

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense,GRU,Conv1D
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from tensorflow.keras.optimizers import Adam

In [ ]:
def Bi_LSTM(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model
def Bi_GRU(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(GRU(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model
def Bi_LSTM_CNN(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  model = Conv1D(200,3,padding = "same",activation = "relu")(model)
  model = Conv1D(200,4,padding = "same",activation = "relu")(model)
  model = Conv1D(200,5,padding = "same",activation = "relu")(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model
def Bi_GRU_CNN(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(GRU(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  model = Conv1D(200,3,padding = "same",activation = "relu")(model)
  model = Conv1D(200,4,padding = "same",activation = "relu")(model)
  model = Conv1D(200,5,padding = "same",activation = "relu")(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model

##Training trên bộ AI

In [ ]:
BiLSTM_a = Bi_LSTM(num_words_a,num_tags_a,max_len)
BiGRU_a = Bi_GRU(num_words_a,num_tags_a,max_len)
BiLSTMCNN_a = Bi_LSTM_CNN(num_words_a,num_tags_a,max_len)
BiGRUCNN_a = Bi_GRU_CNN(num_words_a,num_tags_a,max_len)
history1_a = BiLSTM_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history1_a = BiLSTM_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history2_a = BiGRU_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history3_a = BiLSTMCNN_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history4_a = BiGRUCNN_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_38 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_37 (Embedding)    (None, 100, 100)          558800    
                                                                 
 spatial_dropout1d_37 (Spati  (None, 100, 100)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_37 (Bidirecti  (None, 100, 200)         160800    
 onal)                                                           
                                                                 
 time_distributed_37 (TimeDi  (None, 100, 29)          5829      
 stributed)                                                      
                                                          

##Kết quả trên bộ AI

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def score(model, x_test, y_test,tags):
  y_predict = model.predict(x_test)
  y_predict = np.argmax(y_predict, axis=-1)
  y_predict = y_predict.reshape(1,-1)[0]
  y_true = y_test.reshape(1,-1)[0]
  entity_predict = [tags[i] for i in y_predict]
  entity_true = [tags[i] for i in y_true]
  f1_ma = f1_score(entity_true,entity_predict, average="macro")
  f1_mi= f1_score(entity_true,entity_predict, average="micro")
  print("f1 macro: ", f1_ma)
  print("f1 micro: ", f1_mi)

In [ ]:
score(BiLSTM_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.1455312592346394
f1 micro:  0.9353333333333333


In [ ]:
score(BiLSTMCNN_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.24855813808011604
f1 micro:  0.9344444444444444


In [ ]:
score(BiGRU_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.10900272554925455
f1 micro:  0.9311111111111111


In [ ]:
score(BiGRUCNN_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.28866342773994297
f1 micro:  0.9388888888888889


##Training trên bộ music

In [ ]:
BiLSTM_m = Bi_LSTM(num_words_m,num_tags_m,max_len)
BiGRU_m = Bi_GRU(num_words_m,num_tags_m,max_len)
BiLSTMCNN_m = Bi_LSTM_CNN(num_words_m,num_tags_m,max_len)
BiGRUCNN_m = Bi_GRU_CNN(num_words_m,num_tags_m,max_len)
history1_m = BiLSTM_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history2_m = BiLSTMCNN_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history3_m = BiGRU_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history4_m = BiGRUCNN_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_13 (Embedding)    (None, 100, 100)          743100    
                                                                 
 spatial_dropout1d_13 (Spati  (None, 100, 100)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_13 (Bidirecti  (None, 100, 200)         160800    
 onal)                                                           
                                                                 
 time_distributed_13 (TimeDi  (None, 100, 27)          5427      
 stributed)                                                      
                                                          

##Kết quả trên bộ Music

In [ ]:
score(BiLSTM_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.1309349313459796
f1 micro:  0.8625


In [ ]:
score(BiLSTMCNN_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.44808364886869273
f1 micro:  0.9125


In [ ]:
score(BiGRU_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.19263047306319656
f1 micro:  0.8810416666666665


In [ ]:
score(BiGRUCNN_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.5118107930748979
f1 micro:  0.9166666666666666


##Training trên bộ Politics

In [ ]:
BiLSTM_p = Bi_LSTM(num_words_p,num_tags_p,max_len)
BiGRU_p = Bi_GRU(num_words_p,num_tags_p,max_len)
BiLSTMCNN_p = Bi_LSTM_CNN(num_words_p,num_tags_p,max_len)
BiGRUCNN_p = Bi_GRU_CNN(num_words_p,num_tags_p,max_len)
history1_p = BiLSTM_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history2_p = BiLSTMCNN_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history3_p = BiGRU_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history4_p = BiGRUCNN_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_17 (Embedding)    (None, 100, 100)          952800    
                                                                 
 spatial_dropout1d_17 (Spati  (None, 100, 100)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_17 (Bidirecti  (None, 100, 200)         160800    
 onal)                                                           
                                                                 
 time_distributed_17 (TimeDi  (None, 100, 19)          3819      
 stributed)                                                      
                                                          

##Kết quả trên bộ Politics

In [ ]:
score(BiLSTM_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.3099826323868402
f1 micro:  0.928


In [ ]:
score(BiLSTMCNN_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.5225757738721736
f1 micro:  0.9461428571428573


In [ ]:
score(BiGRU_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.4117753954257253
f1 micro:  0.9458571428571428


In [ ]:
score(BiGRUCNN_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.5441600343392267
f1 micro:  0.9487142857142857


##Training trên bộ Literature

In [ ]:
BiLSTM_l = Bi_LSTM(num_words_l,num_tags_l,max_len)
BiGRU_l = Bi_GRU(num_words_l,num_tags_l,max_len)
BiLSTMCNN_l = Bi_LSTM_CNN(num_words_l,num_tags_l,max_len)
BiGRUCNN_l = Bi_GRU_CNN(num_words_l,num_tags_l,max_len)
history1_l = BiLSTM_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history2_l = BiLSTMCNN_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history3_l = BiGRU_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history4_l = BiGRUCNN_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_29 (Embedding)    (None, 100, 100)          784700    
                                                                 
 spatial_dropout1d_29 (Spati  (None, 100, 100)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_29 (Bidirecti  (None, 100, 200)         160800    
 onal)                                                           
                                                                 
 time_distributed_29 (TimeDi  (None, 100, 25)          5025      
 stributed)                                                      
                                                          

##Kết quả trên bộ Literature

In [ ]:
score(BiLSTM_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.05750740598473172
f1 micro:  0.8804347826086957


In [ ]:
score(BiLSTMCNN_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.3603265866291133
f1 micro:  0.921304347826087


In [ ]:
score(BiGRU_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.11709456581457918
f1 micro:  0.8982608695652174


In [ ]:
score(BiGRUCNN_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.3729746210297202
f1 micro:  0.9154347826086957


##Training trên bộ Science

In [ ]:
BiLSTM_s = Bi_LSTM(num_words_s,num_tags_s,max_len)
BiGRU_s = Bi_GRU(num_words_s,num_tags_s,max_len)
BiLSTMCNN_s = Bi_LSTM_CNN(num_words_s,num_tags_s,max_len)
BiGRUCNN_s = Bi_GRU_CNN(num_words_s,num_tags_s,max_len)
history1_s = BiLSTM_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history2_s = BiLSTMCNN_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history3_s = BiGRU_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)
history4_s = BiGRUCNN_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_split= 0.05,
    batch_size= 8, 
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_33 (Embedding)    (None, 100, 100)          940200    
                                                                 
 spatial_dropout1d_33 (Spati  (None, 100, 100)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_33 (Bidirecti  (None, 100, 200)         160800    
 onal)                                                           
                                                                 
 time_distributed_33 (TimeDi  (None, 100, 35)          7035      
 stributed)                                                      
                                                          

##Kết quả trên bộ Science

In [ ]:
score(BiLSTM_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.08671184364322905
f1 micro:  0.8953333333333333


In [ ]:
score(BiLSTMCNN_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.3324908461499293
f1 micro:  0.9235


In [ ]:
score(BiGRU_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.15947822801669775
f1 micro:  0.9101666666666667


In [ ]:
score(BiGRUCNN_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.3592691117316034
f1 micro:  0.9218333333333333


##Kết quả dự đoán

In [ ]:
def predict(model, x_test, y_test, words, tags):
  i = np.random.randint(0, x_test.shape[0])
  p = model.predict(np.array([x_test[i]]))
  p = np.argmax(p, axis=-1)
  y_true = y_test[i]
  print("{:15}{:5}\t {}\n".format("Word", "True", "Pred"))
  print("-" *30)
  for w, true, pred in zip(x_test[i], y_true, p[0]):
    print("{:15}{}\t{}".format(words[w-1], tags[true], tags[pred]))

In [ ]:
predict(BiLSTM_a, x_test_a, y_test_a, words_a, tags_a)